# Script for viewing a rollout

In [1]:
import torch
print('Version', torch.__version__)
print('CUDA enabled:', torch.cuda.is_available())

Version 1.7.0+cu101
CUDA enabled: True


In [2]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [3]:
import os
BASE_PATH = '/gdrive/My Drive/colab_files/Final Project/'
os.chdir(BASE_PATH)

In [4]:
!pip install gym[box2d]
!pip3 install -r requirements.txt
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [5]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

In [6]:
from utils.misc import ASIZE, RSIZE, LSIZE
from utils.misc import load_parameters
from utils.misc import flatten_parameters
from utils.misc import RolloutGenerator, transform
from models import VAE, MDRNNCell, Controller

In [7]:
import pyglet
print(pyglet.version)
import argparse
import sys
from os.path import join, exists, abspath
from os import mkdir, unlink, listdir, getpid
from time import sleep
from torch.multiprocessing import Process, Queue
import torch
import cma
from models import Controller
from tqdm import tqdm
import numpy as np
from torchvision import transforms

1.5.0


In [8]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

# This code creates a virtual display to draw game images on. 
# If you are running locally, just ignore it
import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
    !bash ../xvfb start
    %env DISPLAY=:1

In [9]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [10]:
time_limit = 1000
device = "cuda"
pop_size = 4
n_samples = 4
controller = Controller(LSIZE, RSIZE, ASIZE)
parameters = controller.parameters()

# paths to weights for vae, mdrnn, and controller
VAE_WEIGHTS_PATH = "weights/vae_nonorm_b1.pt"
MDRNN_WEIGHTS_PATH = "weights/mdrnn.pt"
CONTROLLER_WEIGHTS_PATH = "ctrl/ctrl_b1.pt"

In [11]:
rg = RolloutGenerator(VAE_WEIGHTS_PATH, MDRNN_WEIGHTS_PATH, device, time_limit)

In [12]:
controller_params = torch.load(CONTROLLER_WEIGHTS_PATH)
rg.controller.load_state_dict(controller_params["state_dict"])

<All keys matched successfully>

In [ ]:
env = wrap_env(gym.make('CarRacing-v0'))
obs = env.reset()
obs = obs[:56]
# This first render is required !
env.render()

hidden = [
    torch.zeros(1, RSIZE).to("cuda")
    for _ in range(2)]

cumulative = 0
i = 0
while True:
    env.render()
    obs = transform(obs).unsqueeze(0).to("cuda")
    action, hidden = rg.get_action_and_transition(obs, hidden)
    obs, reward, done, _ = env.step(action)
    obs = obs[:56]
    cumulative += reward
    if done or i > time_limit:
        print(cumulative)
        break
    i += 1
env.close()
show_video()

Track generation: 1122..1406 -> 284-tiles track
